# Reconocimiento de fonemas usando CTC

In [ ]:
import json
import torch
import torch.utils.data as data
import torchaudio
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import  Dataset, DataLoader
from torchaudio.models.decoder import ctc_decoder # vamos a hacer un decoder greedy, por razones 
                                                  # didácticas, este se usaría si quisiera
                                                  # implementarlo con beam search

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Implementación del dataset y el dataloader

### Dataset
Me convierte los datos crudos para que puedan ser usados por `Dataloader`. Me permite implementar una función `__getitem__()` en la cual leemos los datos y devolvemos por ejemplo el wav y la transcripción de cada dato.

In [ ]:
train_json = 'data/train.json'
test_json = 'data/test.json'
valid_json = 'data/dev.json'

class TimitDataset(Dataset):
    def __init__(self, json_file):
        try:
            with open(json_file, 'r') as f:
                self.datos_json = json.load(f)
        except FileNotFoundError:
            print(f"Error: El archivo {json_file} no se encuentra.")
        # Get a list of all sample IDs (keys in the top-level dictionary)
        self.datos_ids = list(self.datos_json.keys())

    def __len__(self):
        return len(self.datos_json)
    
    def __getitem__(self, idx):
        key = self.datos_ids[idx]
        wavdir = self.datos_json[key]['wav']
        duration = self.datos_json[key]['duration']
        phn = self.datos_json[key]['phn']
        # Load the audio file
        waveform, sample_rate = torchaudio.load(wavdir)
        # Convert waveform to a 1D tensor
        waveform = waveform.squeeze(0)
        # Convert phoneme labels to a tensor
        phn_list = phn.strip().split()
        
        return waveform, phn_list

def collate_fn(batch):
    # El batch es una lista de tuplas: [(dato1,label1), (dato2,label2),...]
    sequences, labels = zip(*batch) # Esto devuelve: 
                                    # sequences = (dato1,dato2,...)
                                    # labels = (label1,label2,...)
    labels = [torch.tensor([ord(c) for c in label]) for label in labels]
    padded_sequences = pad_sequence(sequences, batch_first=True, padding_value=0)
    padded_labels = pad_sequence(torch.tensor(labels), batch_first=True, padding_value= '\n')
    return padded_sequences, padded_labels # Esta es la salida del dataloader
    return padded_sequences, padded_labels
train_ds = TimitDataset(train_json)
test_ds = TimitDataset(test_json)
valid_ds = TimitDataset(valid_json)

train_dl = DataLoader(train_ds, batch_size=8, shuffle=True, collate_fn=collate_fn)
test_dl = DataLoader(train_ds, batch_size=16, shuffle=False, collate_fn=collate_fn)
valid_dl = DataLoader(train_ds, batch_size=16, shuffle=False, collate_fn=collate_fn)

  
        

In [36]:
i=0
for batch in train_dl:
    sequences, labels = batch
    print("Tamaño del batch:", sequences.size())
    print("Tamaño de las etiquetas:", len(labels))
    i += 1
    if i >= 5:  # Limitar a un solo batch para evitar imprimir demasiado
        break  # Solo para mostrar el primer batch y evitar imprimir demasiado

TypeError: ord() expected a character, but string of length 3 found

In [34]:
a = ['s','r','m']
int(a)

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'list'